In [1]:
import sys

In [2]:
print(sys.executable)

F:\python\Anaconda3\envs\tensorFLow-gpu\python.exe


In [3]:
# 导入所需模块
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.
F:\python\Anaconda3\envs\tensorFLow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\python\Anaconda3\envs\tensorFLow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\python\Anaconda3\envs\tensorFLow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\python\Anaconda3\envs\tensorFLow

In [4]:
# 读取CIFAR-10数据
(x_train_img, y_train_label),(x_test_img, y_test_label) = cifar10.load_data()

In [5]:
# 显示训练和验证数据的shape
print('x_train_img.shape: ', x_train_img.shape, ' y_train_label.shape: ', y_train_label.shape)
print('x_test_img.shape: ', x_test_img.shape, ' y_test_label.shape: ', y_test_label.shape)

x_train_img.shape:  (50000, 32, 32, 3)  y_train_label.shape:  (50000, 1)
x_test_img.shape:  (10000, 32, 32, 3)  y_test_label.shape:  (10000, 1)


In [6]:
# 将features（照片的特征值）标准化
# 标准化可以提高预测精度，并且可以更快收敛
x_train_img_normalize = x_train_img.astype('float32') / 255.0
x_test_img_normalize = x_test_img.astype('float32') / 255.0

In [7]:
# label以一位有效编码进行转换
from keras.utils import np_utils
y_train_label_OneHot = np_utils.to_categorical(y_train_label)
y_test_label_OneHot = np_utils.to_categorical(y_test_label)

### 建立模型

In [8]:
# 1.导入所需模块
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [9]:
model = Sequential()

### 建立卷积层1与池化层1

In [10]:
# 建立卷积层1
model.add(Conv2D(filters=32,
                                kernel_size=(3, 3),
                                input_shape=(32, 32, 3),
                                activation='relu',
                                padding='same'))
# 加入DropOut，避免过度拟合
model.add(Dropout(rate=0.3))

# 【新增加Conv2D 层】
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))

# 建立池化层1，执行缩减采样
model.add(MaxPooling2D(pool_size=(2,2)))

### 建立卷积层2与池化层2

In [11]:
# 建立卷积层2
model.add(Conv2D(filters=64,
                                kernel_size=(3, 3),
                                activation='relu',
                                padding='same'))
# 加入DropOut，避免过度拟合
model.add(Dropout(rate=0.3))

# 【新增加Conv2D 层】
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))

# 建立池化层2，执行缩减采样
model.add(MaxPooling2D(pool_size=(2,2)))

### 建立卷积层3和池化层3

In [12]:
# 建立卷积层3
model.add(Conv2D(filters=128,
                                kernel_size=(3, 3),
                                activation='relu',
                                padding='same'))
# 加入DropOut，避免过度拟合
model.add(Dropout(rate=0.3))

# 【新增加Conv2D 层】
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))

# 建立池化层3，执行缩减采样
model.add(MaxPooling2D(pool_size=(2,2)))

#### 建立神经网络层（平坦层，隐藏层1，隐藏层2，输出层）

**我们建立更宽，更深的神经网络， 加入隐藏层1（2500个神经元）和隐藏层2（1500个神经元）。**

In [13]:
# 平坦层
model.add(Flatten())
model.add(Dropout(0.3))
# 隐藏层1
model.add(Dense(2500, activation='relu'))
model.add(Dropout(0.3))
# 隐藏层2
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.3))
# 输出层
model.add(Dense(10, activation='softmax'))

#### 训练模型

#### 为了增加准确率,执行50个训练周期，这需要花很长时间

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_history = model.fit(x_train_img_normalize, y_train_label_OneHot, validation_split=0.2, epochs=50, batch_size=300, verbose=1)

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 81s 2ms/step - loss: 1.8643 - acc: 0.3034 - val_loss: 1.7372 - val_acc: 0.3697
Epoch 2/50
40000/40000 [==============================] - 70s 2ms/step - loss: 1.4255 - acc: 0.4737 - val_loss: 1.2955 - val_acc: 0.5322
Epoch 3/50
40000/40000 [==============================] - 69s 2ms/step - loss: 1.2249 - acc: 0.5573 - val_loss: 1.1833 - val_acc: 0.5662
Epoch 4/50
40000/40000 [==============================] - 70s 2ms/step - loss: 1.0878 - acc: 0.6105 - val_loss: 1.0088 - val_acc: 0.6537
Epoch 5/50
40000/40000 [==============================] - 69s 2ms/step - loss: 0.9942 - acc: 0.6447 - val_loss: 0.9049 - val_acc: 0.6817
Epoch 6/50
40000/40000 [==============================] - 70s 2ms/step - loss: 0.8949 - acc: 0.6807 - val_loss: 0.8829 - val_acc: 0.6949
Epoch 7/50
40000/40000 [==============================] - 69s 2ms/step - loss: 0.8296 - acc: 0.7029 - val_loss: 0.8450 - val_acc

In [ ]:
# 建立show_train_history显示训练过程
# 可以使用如下方法，读取train_history，以图表显示训练过程。
import matplotlib.pyplot as plt
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.xlabel('Epoch')
    plt.ylabel('Train')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
show_train_history(train_history, 'acc', 'val_acc')

In [ ]:
scores = model.evaluate(x_test_img_normalize, y_test_label_OneHot, verbose=0)
print(scores[1])

## 10.10模型的保存与加载

### 上面程序的训练必须花费很长时间，往往需要数小时。有时还可能因为某些原因导致计算机宕机，这样之前的训练就前功尽弃了，解决的方法是：每次程序执行完成训练后，将模型权重保存一下。下次程序执行训练之前，先加载模型权重，在继续训练

In [ ]:
# 1.设置训练周期
# 每次训练的周期不要太多，下面设置为5
train_history = model.fit(x_train_img_normalize, y_train_label_OneHot, validation_split=0.2, epochs=5, batch_size=128, verbose=1)

In [ ]:
# 2.在执行训练之前加载模型权重
try:
    model.load_weights("saveModel/cifarCnnModel.h5")
    print('模型加载成功！继续训练模型')
except:
    print('加载模型失败！开始训练一个模型')

### 从以上执行结果可知，因为第一次执行尚未保存模型权重，所以会显示“加载模型失败！开始训练一个模型”。

In [ ]:
# 在程序的最后保存模型权重
# 将这次执行5个训练周期的结果使用model.save_weights保存在文件中。
model.save_weights('saveModel/cifarCnnModel.h5')
print('save Model to dist')

In [ ]:
# 第二次执行程序
# 第二次执行之前，同样先试用 model.load_weights 加载模型权重
try:
    model.load_weights('saveModel/cifarCnnModel.h5')
    print('模型加载成功！继续训练模型')
except:
    print('加载模型失败，开始训练一个新模型')

### 从以上执行结果可知，因为第二次执行会加载之前保存的模型权重，所以会显示“模型加载成功！继续训练模型”，这样就可以接着第一次训练的结果继续训练

***

## 10.11 结论

### 在本章节中，我们介绍了使用keras建立卷积神经网络识别 CIFAR-10图像数据。后续章节我们将以 _<u>多层感知器模型</u>_ 来预测泰坦尼克号乘客的生存率